In [ ]:

import os
import traceback
import pyproj
# N.B.: you'll need to set this up for your own system
pyproj.datadir.set_data_dir("/home/ben/mambaforge/envs/transition-points/share/proj")
from tqdm import tqdm
from shapely import LineString, MultiPolygon, Polygon
import shapely
import geopandas as gpd

%load_ext autoreload 
%autoreload 2

import Intersections
import KmlReader as Kr
import Conversions
import algo
import ShpConverter
import FileManager as Fm
import AscReq
import Warnings
import gui
import numpy as np
%load_ext autoreload
%autoreload 2

In [ ]:
"""
Function orchestrates the entire processing pipeline.

Steps:
    1. Gather user inputs with gui
    2. Processes mask file (KML or Shapefile)
    3. Generate new land mask by combining or modifying the existing one
    4. Read and processes the RGTs from Cycle 12
    5. Generates segments and assigns points
    6. Validate and filter points
    7. Writes final points to a CSV file
    8. Generates transition errors and warnings
    9. Downloads final files to specified destination
"""

#gui.run()

# Sets vars based off user inputs

### GUI inputs here, now hardcoded
off_pointing = True #if gui.mask_region_type == 'Off-Pointing' else False
transition_csv_path = "RGT_transition_locations_V2.0 1.csv"#gui.transition_csv_path
threshold_kilometers = 100#gui.threshold_kilometers
save_file=os.path.join('assets', 'Antarctic_10km_plus_Greenland.csv')


# Conditional import based on type of mask
if off_pointing:
    import CombinePointGenerator as Pg
else:
    import PointerGenerator as Pg

Pg.MIN_TRANSITION_DIST = threshold_kilometers

# using geopandas should mean that we don't need to worry whether the input is a shapefile or a kml
#mask_gcs_coords = Kr.parse_mask(mask_filepath)
gdf_AA = gpd.read_file('assets/Antarctica_100km.geojson')
gdf_GL = gpd.read_file('GL_offpointing.kml')

mask_geom = shapely.unary_union([gdf_AA.geometry[0], gdf_GL.geometry[0]])

#if len(gdf.geometry) ==1:
#    mask_geom = gdf.geometry[0]
#else:
#    mask_geom = shapely.unary_union([gdf.geometry])

try:
    mask_gcs_coords=[list(jj.exterior.coords) for jj in mask_geom.geoms]
except AttributeError:
    mask_gcs_coords=[list(jj.exterior.coords) for jj in [mask_geom]]

# Converts coordinates to cartesian to prepare for shapely processing
mask_polygons_cart = [Polygon(Conversions.gcs_list_to_cartesian(coords)) for coords in mask_gcs_coords]
mask_multipolygon = shapely.make_valid(MultiPolygon(mask_polygons_cart))

#land_gcs_coords = Kr.parse_mask(os.path.join('assets', 'land_mask.kml'))
#land_polygon_cart = [Polygon(Conversions.gcs_list_to_cartesian(coordinates)) for coordinates in land_gcs_coords]

land_polygon = shapely.unary_union([gpd.read_file(os.path.join('assets', 'land_mask.kml')).geometry])
land_gcs_coords = [list(jj.exterior.coords) for jj in land_polygon.geoms]
land_polygon_cart = [Polygon(Conversions.gcs_list_to_cartesian(coordinates)) for coordinates in land_gcs_coords]
land_multipolygon = shapely.make_valid(MultiPolygon(land_polygon_cart))

# Combines mask and land mask or modifies land mask based on type of pointing required
if not off_pointing:
    new_land_multipolygon = Intersections.modify_land_mask(land_multipolygon, mask_multipolygon)  # RETURNS back GCS
else:
    new_land_multipolygon = Intersections.combine_land_mask(land_multipolygon, mask_multipolygon)  # RETURNS back GCS

# Validates the new land mask
new_land_cart = [Polygon(Conversions.gcs_list_to_cartesian(polygon.exterior.coords))
                 for polygon in new_land_multipolygon.geoms]
new_land_final_multi = shapely.make_valid(MultiPolygon(new_land_cart))

# Creates a list of all the RGT files
dir_name = os.path.join('assets', 'IS2_RGTs_cycle12_date_time')
ext = '.kml'

file_list = []

for file in os.listdir(dir_name):
    if file.endswith(ext):
        file_list.append(file)

file_list.sort()  # Requires consistent file names

# Initializes and populates a dictionary containing all the original Transition Points for each RGT
points_dict = {}
for i in range(1, 1388):
    points_dict[i] = []
points_dict = Fm.read_csv(transition_csv_path, points_dict)

# Processes all 1387 RGTs, segments them, and modifies Transition Points
rgt = 1
start_latitude = 0.0279589282518  # Required for sorting
start_longitude = -0.131847178124  # Required for sorting
#pbar = tqdm(total=1387, desc='Processing', colour='blue')  # Initializes console status bar

for file in file_list:
    # Reads RGT coordinates from file and converts that into a cartesian LineString
    #orbit_gcs = Kr.get_coordinates_from_kml(os.path.join(dir_name, file))
    orbit_gcs = (gpd.read_file(os.path.join(dir_name, file), layer=file).geometry[0].coords)
    orbit_cart = Conversions.gcs_list_to_cartesian(orbit_gcs)
    orbit_line = shapely.make_valid(LineString(orbit_cart))

    # Splits RGT into segments if portions cross the anti meridian
    segments = Pg.split_anti_meridian(LineString(Conversions.cartesian_list_to_gcs(list(orbit_line.coords))))

    if len(segments) == 1:  # This means that the RGT did not cross the antimeridian
        if off_pointing:
            segments_clean = Pg.segmentation(mask_multipolygon, new_land_final_multi, orbit_line)
        else:
            segments_clean = Pg.segmentation(new_land_final_multi, orbit_line)

        # Cleans segments and assigns points to them
        segments_clean = Pg.remove_insignificant_segments(segments_clean)
        segments_clean = Pg.merge_touching_segments(segments_clean)
        segments_clean = Pg.sort_segments_by_coordinates(segments_clean,
                                                         Conversions.gcs_to_cartesian(start_latitude,
                                                                                      start_longitude))
        segments_clean = Pg.remove_segments_under_thresh(segments_clean)
        segments_clean = Pg.merge_rgt_ocean(segments_clean)
        segments_clean = Pg.merge_corresponding_segments(segments_clean)
        segments_clean = Pg.assign_points(rgt, points_dict, segments_clean)

        segments_clean = algo.validate_points(segments_clean, rgt)  # Validates the points

        # Stores the points in the points_dict dictionary
        points_dict[rgt] = []
        for segment in segments_clean:
            if len(segment.points) != 0:
                for point in segment.points:
                    points_dict[rgt].append(point)

    else:
        segments_combined = []  # list of all segments
        for i in range(len(segments)):
            if not off_pointing:
                segments_clean = Pg.segmentation(mask_multipolygon, new_land_final_multi,
                                                 LineString(
                                                     Conversions.gcs_list_to_cartesian(list(segments[i].coords))))
            else:
                segments_clean = Pg.segmentation(new_land_final_multi,
                                                 LineString(Conversions.gcs_list_to_cartesian(
                                                     list(segments[i].coords))))

            # Cleans segments and assigns points to them
            segments_clean = Pg.remove_insignificant_segments(segments_clean)
            segments_clean = Pg.merge_touching_segments(segments_clean)
            segments_clean = Pg.sort_segments_by_coordinates(segments_clean,
                                                             Conversions.gcs_to_cartesian(start_latitude,
                                                                                          start_longitude))
            segments_clean = Pg.remove_segments_under_thresh(segments_clean)
            segments_clean = Pg.merge_rgt_ocean(segments_clean)

            segments_combined.extend(segments_clean)

            coordinates = Conversions.cartesian_to_gcs(list(segments_combined[-1].line_string.coords)[-1][0],
                                                       list(segments_combined[-1].line_string.coords)[-1][1])
            start_longitude = - coordinates[0]
            start_latitude = coordinates[1]

        # Merges segments split by antimeridian
        segments_combined = Pg.merge_corresponding_segments(segments_combined)
        segments_combined = Pg.assign_points(rgt, points_dict, segments_combined)

        segments_combined = algo.validate_points(segments_combined, rgt)  # Validates the points

        # Stores the points in the points_dict dictionary
        points_dict[rgt] = []
        for segment in segments_combined:
            if len(segment.points) != 0:
                for point in segment.points:
                    points_dict[rgt].append(point)

    # Generates Ascending Requirements
    AscReq.generate_asc_req(orbit_gcs, points_dict[rgt])  # TODO I moved, double check it works

    rgt += 1
    cart_coords = orbit_gcs[-1]
    gcs_coords = cart_coords[0], cart_coords[1]
    start_longitude = gcs_coords[0]
    start_latitude = gcs_coords[1]
    if np.mod(file_list.index(file), 50)==0:
        print(f'{file_list.index(file)} finished out of 1387')

# Filters out points that are unnecessary

# DEBUG THESE
Pg.remove_twilight_points(points_dict)
Pg.remove_duplicate_points(points_dict)
Pg.remove_extra_endpoints(points_dict)
Pg.remove_points_under_threshold(points_dict, Pg.MIN_TRANSITION_DIST)

# Writes new points to file
Fm.write_csv(save_file, points_dict)

# Generates errors and warnings
transition_errors = Pg.generate_transition_errors(points_dict)
Warnings.generate_warnings(transition_errors, Pg.significant_rgts_under_thresh, points_dict, Pg.MIN_TRANSITION_DIST)

# Downloads Files
#Fm.download_files(gui.files_destination)


In [ ]:
# read back output and plot:
new_points=gpd.read_file(save_file)
coords = [*map(float, new_points.lon)], [*map(float, new_points.lat)]
new_points_cart = Conversions.gcs_list_to_cartesian(np.array(coords).T)


from shapely import plotting
import matplotlib.pyplot as plt
plt.figure()
shapely.plotting.plot_polygon(land_multipolygon, add_points=False)
shapely.plotting.plot_polygon(mask_multipolygon, add_points=False, color='red')
plt.plot(*np.c_[new_points_cart].T,'.')